In [14]:
import requests

# URL to fetch data
url = "https://api.cellosaurus.org/search/cell-line?q=ko52"

# Send a GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON response


In [41]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the script tag containing the desired identifier
    script_tags = soup.find_all('script')
    for script in script_tags:
        # Look for the specific identifier in the script content
        if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
            # Extract the full line containing the scatterPlot data
            full_line = re.search(r"var plot = \$\('#celline_prio_leukemia_LAML'\)\.scatterPlot\((.*?)\);", script.text, re.DOTALL)
            # if full_line:
            full_line_text = full_line.group(1)  # Extract the full JSON-like data
            cell_lines = re.findall(r'"name":"(.*?)"', full_line_text)
            #Strip trailers
            cell_lines = [x.split(' ')[0].replace('\\','') for x in cell_lines]
            print('All Leukemia Cell Lines in Protein Atlas:',cell_lines)  


All Leukemia Cell Lines in Protein Atlas: ['Kasumi-6', 'MOLM-6', 'KO52', 'MUTZ-3', 'GDM-1', 'Kasumi-1', 'Mono-Mac-1', 'OCI-AML-5', 'KG-1', 'OCI-AML-2', 'SKM-1', 'SIG-M5', 'NOMO-1', 'P31/FUJ', 'AML-193', 'Mono-Mac-6', 'UKE-1', 'EoL-1', 'CMK', 'Loucy', 'U-937', 'ME-1', 'THP-1', 'MOLT-13', 'NALM-1', 'HNT-34', 'NB4', 'HL-60', 'MOLT-16', 'PLB-985', 'MHH-CALL-2', 'EM-2', 'MOLM-13', 'SET-2', 'MHH-CALL-4', 'MV4-11', 'SEM', 'PL-21', 'SKNO-1', 'HEL', 'LAMA-84', 'M-07e', 'MHH-CALL-3', 'KCL-22', 'TF-1', 'Kasumi-2', 'Ku812', 'KE-37', 'CML-T1', 'ML-2', 'MOLM-16', 'NCO2', 'SUP-B15', 'HEL', 'JK-1', 'P12-Ichikawa', 'NALM-19', 'PF-382', 'RPMI-8402', 'MOLT-3', 'OCI-AML-3', 'MUTZ-5', 'HMC-1', 'F-36P', 'TALL-1', 'CCRF-SB', 'RCH-ACV', 'Peer', 'JURKAT', 'JURL-MK1', 'Jurkat', 'KOPN-8', 'JVM-3', 'NALM-6', 'REH', 'ALL-SIL', 'MEC-1', 'DND-41', 'BDCM', 'BV-173', 'OCI-M1', 'MEG-01', 'MOLT-4', 'SUP-T11', 'HPB-ALL', 'SUP-T1', 'JM-1', '697', 'OCI-M2', 'RS4;11', 'K-562', 'KYO-1', 'HAP1']


In [40]:
import pandas as pd
samples = pd.read_csv('/Users/Surya/Downloads/screen-tsv-download.tsv', header=None, delimiter='\t')
samples.columns = ['cell_type','tissue']
leuk_samples = samples[samples['tissue'].str.contains('bone', case=False, na=False)]
print("There are", len(leuk_samples), "samples taken from a bone element/bone marrow")


There are 34 samples taken from a bone element/bone marrow


In [37]:
import requests

def extract_name_list_synonyms(data):
    """
    Extract the name list and synonyms from the API response.

    Args:
        data (dict): Parsed JSON response from the Cellosaurus API.

    Returns:
        dict: A dictionary containing identifiers and their corresponding synonyms.
    """
    results = {}
    # Iterate over each cell line in the 'cell-line-list'
    for cell_line_entry in data.get('cell-line-list', []):
        # Find the identifier
        identifier = None
        synonyms = []
        for name_entry in cell_line_entry.get('name-list', []):
            if name_entry['type'] == 'identifier':
                identifier = name_entry['value']
            elif name_entry['type'] == 'synonym':
                synonyms.append(name_entry['value'])

        # Store the identifier and its synonyms
        if identifier == query:
            results[identifier] = synonyms

    return results


# Fetch data from the API for a specific cell line
for query in cell_lines[0:5]:
    url = "https://api.cellosaurus.org/search/cell-line?q=" + query
    response = requests.get(url)

    if response.status_code == 200:
        # Parse JSON response
        api_data = response.json()
        # Extract synonyms
        synonyms_data = extract_name_list_synonyms(api_data.get('Cellosaurus', {}))

        # Print the results
        for identifier, synonyms in synonyms_data.items():
            print(f"Identifier: {identifier}")
            print(f"Synonyms: {', '.join(synonyms) if synonyms else 'No synonyms found'}")
            print("-" * 40)
    else:
        print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")


Identifier: Kasumi-6
Synonyms: KASUMI-6, KASUMI6
----------------------------------------
Identifier: MOLM-6
Synonyms: MOLM6
----------------------------------------
Identifier: KO52
Synonyms: K052, K-052
----------------------------------------
Identifier: MUTZ-3
Synonyms: Mutz-3, MUTZ3, Mutz3, Menschliche Und Tierische Zellkulture-3
----------------------------------------
Identifier: GDM-1
Synonyms: GDM1
----------------------------------------


In [50]:
import pandas as pd
import requests

def extract_name_list_synonyms(data, query):
    """
    Extract the name list and synonyms for a specific query from the API response.

    Args:
        data (dict): Parsed JSON response from the Cellosaurus API.
        query (str): The cell line query string.

    Returns:
        dict: A dictionary containing the identifier and its corresponding synonyms.
    """
    results = {}
    # Iterate over each cell line in the 'cell-line-list'
    for cell_line_entry in data.get('cell-line-list', []):
        # Find the identifier
        identifier = None
        synonyms = []
        for name_entry in cell_line_entry.get('name-list', []):
            if name_entry['type'] == 'identifier':
                identifier = name_entry['value']
            elif name_entry['type'] == 'synonym':
                synonyms.append(name_entry['value'])

        # Store the identifier and its synonyms if it matches the query
        if identifier == query:
            results[identifier] = synonyms

    return results

def find_matching_rows(cell_lines, samples_df):
    """
    Matches rows in the samples DataFrame based on cell line names and synonyms.

    Args:
        cell_lines (list): List of cell lines to query.
        samples_df (pd.DataFrame): DataFrame containing "cell_type" and "tissue" columns.

    Returns:
        pd.DataFrame: DataFrame with matched rows and an additional column for the identifier.
    """
    matched_rows = []

    for query in cell_lines:
        url = "https://api.cellosaurus.org/search/cell-line?q=" + query
        response = requests.get(url)

        if response.status_code == 200:
            # Parse JSON response
            api_data = response.json()
            # Extract synonyms
            synonyms_data = extract_name_list_synonyms(api_data.get('Cellosaurus', {}), query)

            for identifier, synonyms in synonyms_data.items():
                # Combine identifier and synonyms into one list of query strings
                query_strings = [identifier] + synonyms

                # Match rows where cell_type contains any of the query strings
                for _, row in samples_df.iterrows():
                    if any(query_string in row['cell_type'] for query_string in query_strings):
                        matched_row = row.to_dict()
                        matched_row['matched_identifier'] = identifier
                        matched_rows.append(matched_row)
        else:
            print(f"Failed to fetch data for {query}. HTTP Status Code: {response.status_code}")

    # Create a new DataFrame from the matched rows
    return pd.DataFrame(matched_rows)

# Example cell lines list
cell_lines = cell_lines[0:5]
cell_lines.append('Jurkat')

# Example samples DataFrame
samples_df = samples

# Find matching rows
matched_df = find_matching_rows(cell_lines, samples_df)

# Display the output DataFrame
print(matched_df)


            cell_type        tissue matched_identifier
0  Jurkat; Clone E6-1  bodily fluid             Jurkat


In [64]:
import pandas as pd
import requests

def extract_name_list_synonyms(data, query):
    """
    Extract the name list and synonyms for a specific query from the API response.

    Args:
        data (dict): Parsed JSON response from the Cellosaurus API.
        query (str): The cell line query string.

    Returns:
        dict: A dictionary containing the identifier and its corresponding synonyms.
    """
    results = {}
    # Iterate over each cell line in the 'cell-line-list'
    for cell_line_entry in data.get('cell-line-list', []):
        # Find the identifier
        identifier = None
        synonyms = []
        for name_entry in cell_line_entry.get('name-list', []):
            if name_entry['type'] == 'identifier':
                identifier = name_entry['value']
            elif name_entry['type'] == 'synonym':
                synonyms.append(name_entry['value'])

        # Store the identifier and its synonyms if it matches the query
        if identifier.lower() == query.lower():
            results[identifier] = synonyms

    return results

def find_matching_rows(cell_lines, samples_df):
    """
    Matches rows in the samples DataFrame based on cell line names and synonyms, case-insensitively.

    Args:
        cell_lines (list): List of cell lines to query.
        samples_df (pd.DataFrame): DataFrame containing "cell_type" and "tissue" columns.

    Returns:
        pd.DataFrame: DataFrame with matched rows and an additional column for the identifier.
    """
    matched_rows = []

    # Normalize cell_lines to lowercase for case-insensitive matching
    normalized_cell_lines = [line.lower() for line in cell_lines]

    for query in normalized_cell_lines:
        url = f"https://api.cellosaurus.org/search/cell-line?q={query}"
        response = requests.get(url)

        if response.status_code == 200:
            # Parse JSON response
            api_data = response.json()
            # Extract synonyms
            synonyms_data = extract_name_list_synonyms(api_data.get('Cellosaurus', {}), query)

            for identifier, synonyms in synonyms_data.items():
                # Combine identifier and synonyms into one list of query strings
                query_strings = [identifier.lower()] + [synonym.lower() for synonym in synonyms]

                # Match rows where cell_type contains any of the query strings (case-insensitively)
                for _, row in samples_df.iterrows():
                    if any(query_string in row['cell_type'].lower() for query_string in query_strings):
                        matched_row = row.to_dict()
                        matched_row['matched_identifier'] = identifier
                        matched_rows.append(matched_row)
        else:
            print(f"Failed to fetch data for {query}. HTTP Status Code: {response.status_code}")

    # Create a new DataFrame from the matched rows
    return pd.DataFrame(matched_rows)


In [65]:
cell_lines = cell_lines[0:5]
cell_lines.append('jurkat e6.1')

# Example samples DataFrame
samples_df = samples

# Find matching rows
matched_df = find_matching_rows(cell_lines, samples_df)

# Display the output DataFrame
print(matched_df)

            cell_type        tissue matched_identifier
0  Jurkat; Clone E6-1  bodily fluid        Jurkat E6.1


# Get the Disease Fetcher

In [88]:

url = f"https://api.cellosaurus.org/search/cell-line?q=" + identifier
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # print(data)
    for cell_line_entry in data.get('cell-line-list', []):
        # Debug: Check if we are processing the correct cell line
        # print('a')
        print(cell_line_entry)
        print(f"Processing identifier: {identifier}")
        print(f"Cell line entry: {cell_line_entry.get('name-list', [])}")

        # Look for the first instance of the disease-list and retrieve the first "label"
        disease_list = cell_line_entry.get('disease-list', [])
        if disease_list:
            print(f"Disease list found: {disease_list}")
            if 'label' in disease_list[0]:
                # return disease_list[0]['label']
                print(disease_list[0]['label'])
        else:
            print(f"No disease-list found for identifier: {identifier}")
else:
    print(f"Failed to fetch data for {identifier}. HTTP Status Code: {response.status_code}")
print('?')



?


In [105]:
# Assuming data is your dictionary
cell_line_list = data['Cellosaurus'].get('cell-line-list')
print(type(cell_line_list))
print(cell_line_list)
#


<class 'list'>
[{'accession-list': [{'type': 'primary', 'value': 'CVCL_1230'}], 'age': '66Y', 'category': 'Cancer cell line', 'child-list': [], 'comment-list': [{'category': 'Omics', 'value': 'Transcriptome analysis by RNAseq'}, {'category': 'Omics', 'value': 'Transcriptome analysis by microarray'}, {'category': 'Omics', 'value': 'SNP array analysis'}, {'category': 'Omics', 'value': 'DNA methylation analysis'}, {'category': 'Omics', 'value': 'Deep quantitative proteome analysis'}, {'category': 'Omics', 'value': 'Deep exome analysis'}, {'category': 'Population', 'value': 'Caucasian'}, {'category': 'Part of', 'value': 'TCGA-110-CL cell line panel'}, {'category': 'Part of', 'value': 'COSMIC cell lines project'}, {'category': 'Part of', 'value': 'Cancer Dependency Map project (DepMap) (includes Cancer Cell Line Encyclopedia - CCLE)'}], 'created': '2012-04-04', 'derived-from-site-list': [{'site': {'site-type': 'In situ', 'value': 'Peripheral blood', 'xref-list': [{'accession': 'UBERON_00001

In [106]:
# Assuming `data` contains your dictionary
cell_line_list = data['Cellosaurus'].get('cell-line-list')  # Access the list

# Access the first element of the list (a dictionary)
if isinstance(cell_line_list, list) and len(cell_line_list) > 0:
    first_entry = cell_line_list[0]  # First element in the list

    # Check if 'disease-list' exists in the first dictionary
    if 'disease-list' in first_entry:
        disease_list = first_entry['disease-list']  # Extract the 'disease-list'
        print("Disease List:", disease_list)
else:
    print("Cell line list is empty or not a list.")


Disease List: [{'accession': 'C9154', 'category': 'Medical resources', 'database': 'NCIt', 'label': 'Adult acute myeloid leukemia', 'url': 'https://ncit.nci.nih.gov/ncitbrowser/ConceptReport.jsp?dictionary=NCI_Thesaurus&code=C9154'}, {'accession': 'Orphanet_519', 'category': 'Medical resources', 'database': 'ORDO', 'label': 'Acute myeloid leukemia', 'url': 'https://www.ebi.ac.uk/ols4/ontologies/ordo/classes?iri=http://www.orpha.net/ORDO/Orphanet_519'}]


In [31]:
query = 'bob'
url = "https://api.cellosaurus.org/search/cell-line?q=" + query

In [32]:
url

'https://api.cellosaurus.org/search/cell-line?q=bob'

In [54]:
cell_lines

['Kasumi-6', 'MOLM-6', 'KO52', 'MUTZ-3', 'GDM-1', 'JURKAT']